## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

В данном исследовании представлены таблицы с анализом данных по заемщикам.<br><br>
Исследование разделим на несколько частей.<br><br>
<a href='#section1'>1. Изучение общей информации</a><br><br>
<a href='#section2'>2. Подготовка данных<br><br>
<a href='#section3'>3. Лемматизация<br><br>
<a href='#section4'>4. Категоризация данных<br><br>
<a href='#section5'>5. Определение зависимости между наличием детей и возвратом кредита в срок<br><br>
<a href='#section6'>6. Определение зависимости между семейным положением и возвратом кредита в срок<br><br>
<a href='#section7'>7. Определение зависимости между уровнем дохода и возвратом кредита в срок<br><br>
<a href='#section8'>8. Определение зависимости между целями займа и возвратом кредита в срок<br><br>
<a href='#section9'>9. Общий вывод


## Определение влияния факторов заёмщиков на возврат кредита

### <a id='section1'>1. Изучение общей информации.</a>

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.info()
display(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


#### Вывод

1). Существуют пропуски в данных в столбцах 'days_employed', 'total_income' (оба содержат количественные данные). Причем, если есть пропуск в столбце 'days_employed', то он также встречается в 'total_income', поскольку если потенциальный заемщик не работал, он, скорее всего, и не получал официального дохода. Пропуски заполнены как NaN. Для целей исследования - влияние наличия детей, семейного положения - их можно заполнить медианным значением. Для Цели влияние уровня дохода - лучше исключить.

2). В столбце 'days_employed' есть отрицательные значения. Вероятно, это связано с тем что перерывы между работами идут в минус стажу, и, если перерывов больше то сумма принимает отрицательное значение. У категории пенсионеров меньше явлений отрицательного стажа, возможно это связано с тем что они более ответственно относятся к перерывам в трудовой деятельности. Поскольку цели исследования напрямую не кореллируют с этим столбцом, ему можно придать медианное значение.

3). В столбце 'dob_years' встречаются нереальные значения 0 - их можно заменить медианным значением возраста. Вероятно, это полностью случайный пропуск работников кредитной организации.

4). В столбце 'education' одинаковые слова написаны большими и маленькими буквами- возможно потому, что данные заполняются вручную работниками. Их необходимо стандартизировать.

5). В столбце 'purpose' цели не сгруппированы (например, 'покупка жилья' и 'покупка жилой недвижимости'.

### <a id='section2'>2. Подготовка данных</a>

#### Обработка пропусков

In [3]:
data['days_employed'].fillna(data['days_employed'].median(), inplace=True)
data['total_income'].fillna(data['total_income'].median(), inplace=True)
print(data['education'].value_counts())
print(data['family_status'].value_counts())
print(data['gender'].value_counts())
print(data['income_type'].value_counts())
print(data['purpose'].value_counts())
print(data['dob_years'].value_counts())
data.loc[data['gender'] == 'XNA', 'gender'] = ''
data.loc[data['income_type'] == 'безработный', 'income_type'] = 'другой'
data.loc[data['income_type'] == 'предприниматель', 'income_type'] = 'другой'
data.loc[data['income_type'] == 'студент', 'income_type'] = 'другой'
data.loc[data['income_type'] == 'в декрете', 'income_type'] = 'другой'
data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years'].median()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64
женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
F      14236
M       7288
XNA        1
Name: gender, dtype: int64
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64
свадьба                                   797
на

#### Вывод

Столбцы 'days_employed' и 'total_income' содержат количественные данные, поэтому пропуски в них заполнены медианным значением. Присутствует полностью случайный пропуск в столбце 'gender', заменяем его на пустое значение. Редко встречаемые значения 'income_type' для удобства схлапываем в один под названием 'другой'. Полностью случайную ошибку возраста 0 лет в столбце 'dob_years' заменяем на медианное значение возраста в столбце.

#### Замена типа данных

In [4]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


#### Вывод

Дни и доход удобнее анализировать в целочисленном формате, поэтому поменяем тип с float64 на 'int'.

#### Обработка дубликатов

In [5]:
#явные дубликаты встречаются в столбце 'education'. Приводим все текстовые данные к нижнему регистру.
data['education'] = data['education'].str.lower()
display(data)
#определяем какие значения есть в столбце "Дети"
data['children'].value_counts()
#Мы получили такие значения как -1 и 20. Вероятно, это ошибка оператора и мы заменим их на "1" и "2" соответственно.
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 2
data['children'].value_counts()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,-2113,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,-3112,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

#### Вывод

Обработаны дубликаты в столбцах "Образование" и "Дети".

### <a id='section3'>3. Лемматизация</a>

In [6]:
from pymystem3 import Mystem

In [7]:
m = Mystem()
def make_lemmas(x):
    return m.lemmatize(x)
data['purpose_lemmas'] = data['purpose'].apply(make_lemmas)
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,-8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]"
21521,0,343937,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]"
21522,1,-2113,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]"
21523,3,-3112,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]"


#### Вывод

Создан столбец 'purpose_lemmas', в котором разделены лемматизированы цели. Основные группы целей - автомобиль, недвижимость, свадьба, образование, но их частоту сложно оценить без категоризации.

### <a id='section4'>4. Категоризация данных</a>

In [8]:
#напишем функцию, которая позволит категоризирует данные по целям кредита
def purpose_clean(x):
    lemma = x['purpose_lemmas']
    if 'автомобиль' in lemma:
            return 'Автомобиль'
    if 'свадьба' in lemma:
            return 'Cвадьба'
    if 'недвижимость' in lemma or 'жилье' in lemma:
            return 'Жилье'
    if 'образование' in lemma:
            return 'Образование'
    return 'Другое'
data['purpose_clean'] = data.apply(purpose_clean, axis = 1)
data['purpose_clean'].value_counts()

Жилье          10840
Автомобиль      4315
Образование     4022
Cвадьба         2348
Name: purpose_clean, dtype: int64

In [9]:
#напишем функцию, которая позволит категоризирует данные по уровню дохода
def income_level(x):
    level = x['total_income']
    if level < 150000:
        return 'Низкий уровень дохода'
    elif level < 500000:
        return 'Средний уровень дохода'
    else:
        return 'Высокий уровень дохода'
data['income_level'] = data.apply(income_level, axis = 1)
data['income_level'].value_counts()

Низкий уровень дохода     12341
Средний уровень дохода     8962
Высокий уровень дохода      222
Name: income_level, dtype: int64

#### Вывод

Наиболее распространенными являются такие цели как "Жилье". Затем с более чем двукратным отрывом идут "Автомобиль", "Образование", "Свадьба". Большинство заемщиков имеют доход на уровне прожиточного минимума (низкий).

### <a id='section5'>5. Определение зависимости между наличием детей и возвратом кредита в срок</a>

In [10]:
#Определяем общее количество должников и сохраняем это в переменную total_debt
total_debt = data['debt'].sum()
display(total_debt)

1741

In [11]:
#Определим долю числа должников от общего количества заемщиков
total = data['debt'].count()
print('{:.2%}'.format(total_debt/total))

8.09%


In [12]:
#Для удобства форматируем вывод информации в таблицах
pd.options.display.float_format = '{:.2%}'.format

In [13]:
#Создаем датафрейм для расчетов, в котором присутствуют только должники
data_debt = data[data['debt'] == 1]

In [14]:
#Определяем сколько детей имеют должники (от общего числа должников)
groupby_children = data.groupby('children')[['debt']].sum()
display(groupby_children/total_debt)

,debt
children,
0,61.06%
1,25.56%
2,11.60%
3,1.55%
4,0.23%
5,0.00%


In [15]:
#Определяем какую долю от общего количества заемщиков составляют должники с детьми
display(groupby_children/total)

,debt
children,
0,4.94%
1,2.07%
2,0.94%
3,0.13%
4,0.02%
5,0.00%


In [16]:
#Определяем, какую долю занимают должники с различным количеством детей внутри своей категории
display((data_debt.groupby('children')['debt'].count())/(data.groupby('children')['debt'].count()))

children
0   7.51%
1   9.15%
2   9.48%
3   8.18%
4   9.76%
5    nan%
Name: debt, dtype: float64

#### Вывод

Зависимость между количеством детей и возвратом кредита существует. Почти 5% от всех заемщиков не имеют детей и не возвращают кредит. Однако чем больше детей имеет заемщик, тем выше шанс, что он не отдаст кредит в срок. Для заемщиков с количеством 3+ детей представлено мало данных, поэтому нельзя опираться на выводы по этим группам.

### <a id='section6'> 6. Определение зависимости между семейным положением и возвратом кредита в срок </a>

In [17]:
#Определяем какое семейное положение имеют должники (от общего числа должников)
groupby_family_status = data.groupby('family_status')[['debt']].sum()
display(groupby_family_status/total_debt)

,debt
family_status,
Не женат / не замужем,15.74%
в разводе,4.88%
вдовец / вдова,3.62%
гражданский брак,22.29%
женат / замужем,53.48%


In [18]:
#Определяем какую долю от общего количества заемщиков составляют должники с различным семейным статусом
display(groupby_family_status/total)

,debt
family_status,
Не женат / не замужем,1.27%
в разводе,0.39%
вдовец / вдова,0.29%
гражданский брак,1.80%
женат / замужем,4.33%


In [19]:
#Определяем, какую долю занимают должники с различным семейным статусом внутри своей категории
display((data_debt.groupby('family_status')['debt'].count())/(data.groupby('family_status')['debt'].count()))

family_status
Не женат / не замужем   9.74%
в разводе               7.11%
вдовец / вдова          6.56%
гражданский брак        9.29%
женат / замужем         7.52%
Name: debt, dtype: float64

#### Вывод

Существует некоторая зависимость между возвратом кредита в срок и семейным положением. Более 4% от всех заемщиков не вернули кредит и состоят в официальном браке. Однако заемщики с официальным семейным статусом "Женат\Замужем" чаще возвращают кредит, чем состоящие в статусе "Гражданский брак" или "Не женат/Не замужем". По заемщикам, которые находятся в статусе "В разводе" или "Вдовец/вдова" возможно, недостаточно данных. Но если опираться на те данные, которые мы имеем, можно сделать вывод что они возвращают кредит даже лучше чем те, которые имеют статус "Женат/Замужем". Это можно объяснить тем, что они настолько же ответственны к своим жизненным обязательствам, как и те, кто вступили в официальные семейные отношения, и, вдобавок, испытывают более острую нужду в погашении кредита из-за новых обстоятельств (например, из-за новых обязательств перед детьми, у которых формально остался один родитель).

### <a id='section7'> 7. Определение зависимости между уровнем дохода и возвратом кредита в срок </a>

In [20]:
#Определяем какой уровень дохода имеют должники (от общего числа должников)
groupby_income_level = data.groupby('income_level')[['debt']].sum()
display(groupby_income_level/total_debt)

,debt
income_level,
Высокий уровень дохода,0.80%
Низкий уровень дохода,58.30%
Средний уровень дохода,40.90%


In [21]:
#Определяем какую долю от общего количества заемщиков составляют должники с различным уровнем дохода
display(groupby_income_level/total)

,debt
income_level,
Высокий уровень дохода,0.07%
Низкий уровень дохода,4.72%
Средний уровень дохода,3.31%


In [22]:
#Определяем, какую долю занимают должники с различным уровнем дохода внутри своей категории
display((data_debt.groupby('income_level')['debt'].count())/(data.groupby('income_level')['debt'].count()))

income_level
Высокий уровень дохода   6.31%
Низкий уровень дохода    8.22%
Средний уровень дохода   7.94%
Name: debt, dtype: float64

#### Вывод

Почти 5% от всех заемщиков имеют низкий уровень дохода и не вернули кредит. Чем выше доход заемщика, тем больше вероятность что он вернет кредит в срок.

### <a id='section8'> 8. Определение зависимости между целями займа и возвратом кредита в срок </a> 

In [23]:
#Определяем какую цель кредита имеют должники (от общего числа должников)
groupby_purpose = data.groupby('purpose_clean')[['debt']].sum()
display(groupby_purpose/total_debt)

,debt
purpose_clean,
Cвадьба,10.68%
Автомобиль,23.15%
Жилье,44.92%
Образование,21.25%


In [24]:
#Определяем какую долю от общего количества заемщиков составляют должники с различным целями кредита
display(groupby_purpose/total)

,debt
purpose_clean,
Cвадьба,0.86%
Автомобиль,1.87%
Жилье,3.63%
Образование,1.72%


In [25]:
#Определяем, какую долю занимают должники с различными целями кредита внутри своей категории
display((data_debt.groupby('purpose_clean')['debt'].count())/(data.groupby('purpose_clean')['debt'].count()))

purpose_clean
Cвадьба       7.92%
Автомобиль    9.34%
Жилье         7.21%
Образование   9.20%
Name: debt, dtype: float64

#### Вывод

Почти 4% от всех заемщиков взяли кредит на жилье и не вернули его. Лучше всего возвращают кредиты на жилье и на свадьбу. Более рискованные цели - автомобиль и образование. Вероятно, это связано с тем, что автомобиль и образование - это цели, отвечающие больше интересам индивида. В то время как свадьба и жилье - интересы семьи, и оба члена семьи участвуют в погашении долга.

### <a id='section9'> 9. Общий вывод </a>

Самые большие категории заемщиков - это люди с низким уровнем дохода, состоящие в браке, не имеющие детей и оформляющие кредит на жилье. Из четырех вышеперечисленных категорий наиболее рискованной группой заемщиков являются люди с низким уровнем дохода. Лучше отдают кредиты заемщики состоящие в официальном браке, либо разведенные или вдовцы/вдовы с высоким уровнем дохода, небольшим количеством детей, желающие купить жилье или сыграть свадьбу. Наиболее рискованные факторы для других менее распространенных категорий заемщиков - состоящие в гражданском браке, либо не женаты/не замужем, имеющие много детей, желающие получить кредит на автомобиль или образование.